## This notebook is used to test the FiberSpectrograph CSC
This includes testing that the spectra go to the LFA

In [ ]:
from astropy.io import fits
import astropy.stats
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import csv

from lsst.ts import salobj

In [ ]:
d = salobj.Domain()
# Index 3 is required for the auxtel spectrograph, blue=1, red=2
FiberSpectrograph = salobj.Remote(d, 'FiberSpectrograph', 3)
await FiberSpectrograph.start_task

In [ ]:
tmp = await FiberSpectrograph.evt_heartbeat.next(flush=True, timeout=5)
print(tmp)

In [ ]:
await FiberSpectrograph.cmd_setLogLevel.set_start(level=logging.DEBUG)

In [ ]:
tmp = await FiberSpectrograph.evt_summaryState.aget()
print(salobj.State(tmp.summaryState))

In [ ]:
tmp=await FiberSpectrograph.cmd_standby.start(timeout=15)
print(tmp)

In [ ]:
# Send the CSC offline and shut it down
tmp=await salobj.set_summary_state(FiberSpectrograph, salobj.State.STANDBY, settingsToApply='tucson.yaml', timeout=10)
print(tmp)

In [ ]:
FiberSpectrograph.evt_largeFileObjectAvailable.flush()

tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=1, numExposures=1, type='DARK', source='NONE')
lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
print(lfa)

In [ ]:
FiberSpectrograph.evt_largeFileObjectAvailable.flush()

tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=1, numExposures=1, type='DARK', source='NONE')
lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
print(lfa)

In [ ]:
# download file to a local disk
filename=lfa.url.split('FiberSpectrograph:')[-1]
os.system(f"curl {lfa.url} --output /tmp/{filename}")

In [ ]:
lfa.url.split("FiberSpectrograph:")[-1]

In [ ]:
tmp = await FiberSpectrograph.evt_logMessage.aget(timeout=10)
print(tmp.message)

In [ ]:
tmp.message

In [ ]:
if True:
    plt.ylabel('Intensity (ADU)')
    plt.plot(xdata, ydata,'-g')
    plt.title('Spectral resolution function')
    plt.xlabel('wavelength [nm]')
    plt.show()
    plt.close()

In [ ]:
# Fit a guassian to the profile
mean = xdata[np.argmax(ydata)] # assume the peak we want is the maximum value (NOT ALWAYS THE CASE!)
#mean = 432.   #note this correction
sigma = 2.0        #note this correction
offset=np.median(ydata)

def gaus(x,a,x0,sigma, offset):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))+offset

popt,pcov = curve_fit(gaus,xdata,ydata,p0=[8000.0,mean,sigma, offset])

plt.plot(xdata,ydata,'b+:',label='data')
plt.plot(xdata,gaus(xdata,*popt),'r:',label='fit')
dx=50
plt.xlim([mean-dx,mean+dx])
#plt.ylim([0,2000])
plt.legend()
plt.title('Fit to fiber spectrograph data')
plt.ylabel('Intensity [ADU]')
plt.xlabel('Wavelength [nm]')
plt.show()

print('Amplitude [ADU] = {:.0f}'.format(popt[0]))
print('peak (mean) [nm] = {:.2f}'.format(popt[1]))
print('width (sigma) [nm] = {:0.3f}'.format(popt[2]))
print('FWHM [nm] = {:0.3f}'.format(popt[2]*2.355))

In [ ]:
# Now read in the camera image to look at the signal of the flats

In [ ]:
# Now read in the camera image
# For now just calculate the mediam signal per amp

# Read in image
dir = '/mnt/dmcs/2019-04-03/'
atcamera_fname=data_reader_list[-1]['ATArchiver Image Name']
atcamera_fname='AT_O_20190403_000071-ats-wfs_ccd.fits'

full_file_name=dir+atcamera_fname
print('Opening {}'.format(full_file_name))
hdu_list = fits.open(full_file_name)
#hdu_list.info()
im=hdu_list[12].data # for camera data

In [ ]:
# Check median counts on amplifiers (for flat level testing)
if False:
    # Create an array of the median values
    med_arr=np.zeros(16)
    stddev_arr=np.zeros(16)

    for amp in range(len(hdu_list)-1):
        if amp == 0:
            continue
    #    med_arr[amp]=np.median(hdu_list[amp].data)
        # Calculate mean, median, stddev
        tmp = astropy.stats.sigma_clipped_stats(hdu_list[amp].data, sigma=2)
        med_arr[amp]= tmp[1]
        stddev_arr[amp] = tmp[2]
        print('Median and Stddev values of amplifier {} is {} and {}'.format(amp,med_arr[amp], stddev_arr[amp]))

In [ ]:
plt.imshow(hdu_list[12].data)

In [ ]:
hdu_list[0].header

In [ ]:
tmp[3]